# Install libraries

In [1]:
!pip install scikit-learn
!pip install imbalanced-learn
!pip install mlxtend
!pip install pandas-profiling

                                              0.0/262.6 kB ? eta -:--:--
     -------------                           92.2/262.6 kB 1.7 MB/s eta 0:00:01
     -------------------------------------  256.0/262.6 kB 3.1 MB/s eta 0:00:01
     -------------------------------------- 262.6/262.6 kB 2.3 MB/s eta 0:00:00
                                              0.0/309.8 kB ? eta -:--:--
     -----------------                      143.4/309.8 kB 8.3 MB/s eta 0:00:01
     -------------------------------------- 309.8/309.8 kB 4.8 MB/s eta 0:00:00
                                              0.0/102.4 kB ? eta -:--:--
     ---------------------------------------- 102.4/102.4 kB ? eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
                                              0.0/667.1 kB ? eta -:--:--
     ----------------------------          512.0/667.1 kB 16.2 MB/s eta 0:00:01
     ------------------------------------- 667.1/66

# Import Libraries

In [5]:
!pip install --upgrade --target=%APPDATA%\Python\Python311\site-packages pandas
!pip install --upgrade pandas==1.5.3

import pandas as pd
import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from pandas_profiling import ProfileReport
import warnings
warnings.filterwarnings("ignore")

  Using cached pandas-2.2.0-cp311-cp311-win_amd64.whl (11.6 MB)
  Using cached numpy-1.26.3-cp311-cp311-win_amd64.whl (15.8 MB)
  Using cached python_dateutil-2.8.2-py2.py3-none-any.whl (247 kB)
  Using cached pytz-2023.3.post1-py2.py3-none-any.whl (502 kB)
  Using cached tzdata-2023.4-py2.py3-none-any.whl (346 kB)
  Using cached six-1.16.0-py2.py3-none-any.whl (11 kB)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.20.0 requires pandas<2,>=0.25, but you have pandas 2.2.0 which is incompatible.
tensorflow-intel 2.12.0rc1 requires numpy<1.24,>=1.22, but you have numpy 1.26.3 which is incompatible.


  Using cached pandas-1.5.3-cp311-cp311-win_amd64.whl (10.3 MB)
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.0
    Uninstalling pandas-2.2.0:
      Successfully uninstalled pandas-2.2.0


ImportError: cannot import name 'DataError' from 'pandas.core.base' (C:\Python311\Lib\site-packages\pandas\core\base.py)

# Load Dataset

In [ ]:
names=['age','sex','on_thyroxine','query_on_thyroxine','on_antithyroid_medication',
       'sick','pregnant','thyroid_surgery','I131_treatment','query_hypothyroid','query_hyperthyroid',
      'lithium','goitre','tumor','hypopituitary','psych','TSH_measured','TSH','T3_measured','T3','TT4_measured','TT4','T4U_measured',
      'T4U','FTI_measured','FTI','TBG_measured','TBG','referral_source','classes']

In [ ]:
df=pd.read_csv('allhypo.data',names=names,na_values='?')
df.head()

# Data Pre-processing

## Data manipulation

In [ ]:
#Split classes column
col_mod=df.classes.str.split('.',expand=True)
df[['last','final']]=col_mod
data=df.drop(['classes','final'],axis=1)

In [ ]:
df=data.rename(columns={'last':'classes'})
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.shape

In [ ]:
#Save Dataframe to csv file
df.to_csv('Thyroid_Data.csv')

# Missing values handling

In [ ]:
#Number of Null values in Dataset
df.isnull().sum()

In [ ]:
#Percentage of null values in dataset
df.isnull().sum()/df.shape[0]*100

In [ ]:
#TBG having 100% null values so will drop that column
df=df.drop('TBG',axis=1)

In [ ]:
#age,sex,TSH,T3,TT4,T4U,FTI having null values

In [ ]:
categorical_features=df.select_dtypes(exclude='number')
categorical_features.describe()

In [ ]:
# Checking all the unique values inside categorical features
for feature in categorical_features:
    print('---------------------------------')
    print(f"{feature}:{categorical_features[feature].unique()}")

In [ ]:
#Check for outliers

fig, axes = plt.subplots(3,2, figsize=(18, 10))
  

sns.boxplot(ax=axes[0, 0], data=df, x='classes', y='age')
sns.boxplot(ax=axes[0, 1], data=df, x='classes', y='TSH')
sns.boxplot(ax=axes[1, 0], data=df, x='classes', y='T3')
sns.boxplot(ax=axes[1, 1], data=df, x='classes', y='TT4')
sns.boxplot(ax=axes[2, 0], data=df, x='classes', y='T4U')
sns.boxplot(ax=axes[2, 1], data=df, x='classes', y='FTI')

### There are outliers in dataset so will use median to fill null values in numerical and mode for categorical data

In [ ]:
from sklearn.impute import SimpleImputer

#Handle numerical features
simple_imputer=SimpleImputer(strategy='median')
numerical_missing=pd.DataFrame(simple_imputer.fit_transform(df.select_dtypes(exclude='O')))

#Handle categorical features
cat_imputation=SimpleImputer(strategy='most_frequent')
categorical_missing=pd.DataFrame(cat_imputation.fit_transform(df.select_dtypes(exclude='number')))

numerical_missing.columns=df.select_dtypes(exclude='O').columns
categorical_missing.columns=df.select_dtypes(exclude='number').columns

data=pd.concat([numerical_missing,categorical_missing],axis=1)

In [ ]:
data.head()

In [ ]:
data.isnull().sum()

## Outliers detection and removal

Since all the numerical features not following Normal distribution, will use percentile method to detect the outliers and remove them

In [ ]:
data[data.age>100]

In [ ]:
data=data.drop(data.age.index[1364])

In [ ]:
def outliers_removal(numerical_missing):
    for column in numerical_missing:
        sort=np.sort(numerical_missing[column])
        lower_limit,upper_limit=np.percentile(sort,[0,95])
        detected_outliers=numerical_missing.iloc[np.where((numerical_missing[column]>upper_limit) | (numerical_missing[column]<lower_limit))]
        return detected_outliers


In [ ]:
outliers_data=outliers_removal(data)
outliers_data

In [ ]:
new_df=data.drop(outliers_data.index)
new_df.head()

In [ ]:
new_df.shape

In [ ]:
new_df.columns

In [ ]:
new_df.to_csv('Preprocessed_data.csv',index=False)

In [ ]:
X = new_df.iloc[: , 0:-1]
Y = new_df.iloc[: , -1] 

## Splitting the dataset into the Training set and Test set

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state=1)
X_train.shape,X_test.shape,Y_train.shape,Y_test.shape

## Encoding categorical data

### Encoding the Independent Variable

In [ ]:
ordinal_encoder = OrdinalEncoder()
X_train_cat_encoded = pd.DataFrame(ordinal_encoder.fit_transform(X_train.select_dtypes(exclude='number')))
X_train_cat_encoded.columns = X_train.select_dtypes(exclude='number').columns

In [ ]:
X_test_cat_encoded = pd.DataFrame(ordinal_encoder.transform(X_test.select_dtypes(exclude='number')))
X_test_cat_encoded.columns = X_test.select_dtypes(exclude='number').columns

### Encoding the Dependent Variable

In [ ]:
label_encoder = LabelEncoder()
Y_train_cat_encoded= pd.DataFrame(label_encoder.fit_transform(Y_train))
print(Y_train_cat_encoded.value_counts())
print(Y_train.value_counts())

In [ ]:
Y_test_cat_encoded = pd.DataFrame(label_encoder.transform(Y_test))

## Standardization

In [ ]:
sc = StandardScaler()

X_train_sc=pd.DataFrame(sc.fit_transform(X_train.select_dtypes(exclude='O')))
X_test_sc=pd.DataFrame(sc.transform(X_test.select_dtypes(exclude='O')))

X_train_sc.columns=X_train.select_dtypes(exclude='O').columns
X_test_sc.columns=X_test.select_dtypes(exclude='O').columns


In [ ]:
X_train_final=pd.concat([X_train_sc,X_train_cat_encoded],axis=1)

X_test_final=pd.concat([X_test_sc,X_test_cat_encoded],axis=1)

## Handle imbalanced data

In [ ]:
data['classes'].value_counts()

#### Since the dataset is small, will use over-sampling: SMOTE technique to balance the data

SMOTE (Synthetic Minority Oversampling Technique) consists of synthesizing elements for the minority class, based on those that already exist. It works randomly picking a point from the minority class and computing the k-nearest neighbours for this point. The synthetic points are added between the chosen point and its neighbours.

In [ ]:
X_train_resample,Y_train_resample=SMOTE(random_state=0,k_neighbors=1).fit_resample(X_train_final,Y_train_cat_encoded)
X_test_resample,Y_test_resample=SMOTE(random_state=0,k_neighbors=1).fit_resample(X_test_final,Y_test_cat_encoded)

X_train_resample.shape,X_test_resample.shape,Y_train_resample.shape,Y_test_resample.shape

## Feature Selection

In [ ]:
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from sklearn.ensemble import RandomForestClassifier

In [ ]:
print('Training dataset shape:', X_train_resample.shape, Y_train_resample.shape)
print('Testing dataset shape:', X_test_resample.shape, Y_test_resample.shape)

Y_train_resample_flat = Y_train_resample.to_numpy().ravel()
Y_test_resample_flat = Y_test_resample.to_numpy().ravel()

print('Training dataset shape:', X_train_resample.shape, Y_train_resample_flat.shape)
print('Testing dataset shape:', X_test_resample.shape, Y_test_resample_flat.shape)


In [ ]:
rf = RandomForestClassifier(n_estimators=100, max_depth=5)

forward_fs = sfs(rf , k_features=10,forward=True,floating=False,verbose=2,scoring='accuracy',cv=5)

forward_fs = forward_fs.fit(X_train_resample, Y_train_resample_flat)

In [ ]:
feat_names = list(forward_fs.k_feature_names_)
print(feat_names)

X_train_new=X_train_resample[['age','sex','TSH', 'TT4', 'FTI', 'on_thyroxine', 'on_antithyroid_medication', 'goitre', 'hypopituitary', 'psych', 'T3_measured', 'referral_source']]
X_test_new=X_test_resample[['age','sex','TSH', 'TT4', 'FTI', 'on_thyroxine', 'on_antithyroid_medication', 'goitre', 'hypopituitary', 'psych', 'T3_measured', 'referral_source']]


In [ ]:
rf_model=rf.fit(X_train_new,Y_train_resample_flat)

In [ ]:
def print_Score(clf,x_train,x_test,y_train,y_test,train=True):
    if train:
        pred=clf.predict(x_train)
        clf_report=pd.DataFrame(classification_report(y_train,pred,output_dict=True))
        print("Train Result:\n===============")
        print(f"Accuracy Score:{accuracy_score(y_train,pred)*100:.2f}%")
        print("---------------------------------")
        print(f"Classification Report:\n{clf_report}")
        print("-----------------------------------")
        print(f"Confusion Matrix:\n{confusion_matrix(y_train,pred)}\n")
    elif train==False:
        pred=clf.predict(x_test)
        clf_report=pd.DataFrame(classification_report(y_test,pred,output_dict=True))
        print("Test Result:\n===============")
        print(f"Accuracy Score:{accuracy_score(y_test,pred)*100:.2f}%")
        print("---------------------------------")
        print(f"Classification Report:\n{clf_report}")
        print("---------------------------------")
        print(f"Confusion Matrix:\n{confusion_matrix(y_test,pred)}\n")

In [ ]:
print_Score(rf_model,X_train_new,X_test_new,Y_train_resample_flat,Y_test_resample_flat,train=True)

In [ ]:
print_Score(rf_model,X_train_new,X_test_new,Y_train_resample_flat,Y_test_resample_flat,train=False)

In [ ]:
## Logistic Regression

lr=LogisticRegression(random_state=0,max_iter=10)

lr_model=lr.fit(X_train_new,Y_train_resample_flat)

lr_train_score=print_Score(lr_model,X_train_new,X_test_new,Y_train_resample_flat,Y_test_resample_flat,train=True)
lr_test_score=print_Score(lr_model,X_train_new,X_test_new,Y_train_resample_flat,Y_test_resample_flat,train=False)

In [ ]:
## Navie Bayes Classification

gnb=GaussianNB()

gnb_model=gnb.fit(X_train_new,Y_train_resample_flat)

gnb_train_score=print_Score(gnb_model,X_train_new,X_test_new,Y_train_resample_flat,Y_test_resample_flat,train=True)
gnb_test_score=print_Score(gnb_model,X_train_new,X_test_new,Y_train_resample_flat,Y_test_resample_flat,train=False)

In [ ]:
## Decision Tree

dtc=DecisionTreeClassifier(random_state=0,max_depth=10,min_samples_split=5)
dt_model=dtc.fit(X_train_new,Y_train_resample_flat)

dt_train_score=print_Score(dt_model,X_train_new,X_test_new,Y_train_resample_flat,Y_test_resample_flat,train=True)
dt_test_model=print_Score(dt_model,X_train_new,X_test_new,Y_train_resample_flat,Y_test_resample_flat,train=False)

In [ ]:
## KNN

knn=KNeighborsClassifier()
knn_model=knn.fit(X_train_new,Y_train_resample_flat)

knn_train_score=print_Score(knn_model,X_train_new,X_test_new,Y_train_resample_flat,Y_test_resample_flat,train=True)
knn_test_score=print_Score(knn_model,X_train_new,X_test_new,Y_train_resample_flat,Y_test_resample_flat,train=False)

## EDA
Using Pandas Profiling for EDA

In [ ]:
profile=ProfileReport(data,title="Thyroid Disease Detection",explorative=True)

### Save EDA report to html file

In [ ]:
profile.to_file("EDA Report.html")

In [ ]:
profile.to_notebook_iframe()